# FirecREST Tutorial - Part 2: Job submission

Set your credentials like before and run the cell. You should get status code `200`.

#### Cell 2A

In [ ]:
import json
import requests
import time

# Set you token here
TOKEN='<token>'
FIRECREST_IP = 'http://148.187.98.88:8000'

# This function is useful only to print the response in a nicer way
def handle_response(response):
    print("\nResponse status code:")
    print(response.status_code)
    print("\nResponse headers:")
    print(json.dumps(dict(response.headers), indent=4))
    print("\nResponse json:")
    try:
        print(json.dumps(response.json(), indent=4))
    except json.JSONDecodeError:
        print("-")
        
response = requests.get(
    url=f'{FIRECREST_IP}/status/systems',
    headers={'Authorization': f'Bearer {TOKEN}'}
)

handle_response(response)

## FirecREST Tasks

Before submitting our first job it is important to distinguish between two IDs, slurm’s **job ID** and FirecREST’s **task ID**.

**Slurm’s job ID**
- unique identifier of a slurm job
- it is created by Slurm when the job is submitted
- it can be used to track the state of the job with calls like `squeue` or `sacct`

Every time FirecREST has to interact with slurm it creates a _task_. This _task_ is not necessarily bound to a **job ID**.

**FirecREST’s task ID**
- unique identifier of a FirecREST task
- it is created and updated by FirecREST when the first call regarding this task is created
- it can be used to track the state of the task with the API call we will see later in this section

First let's create locally the job script for our submission. Our job is going to perform a simple SHA-1 calculation of the file we uploaded in the last section.

**Remember to change the output of the script and the location of the `firecrest_input_file.txt` in the cluster so that both paths are in your directory!**

#### Cell 2B

In [ ]:
%%writefile files/firecrest_script.sh
#!/bin/bash

#SBATCH --job-name=test
#SBATCH --output=/home/llama/res.txt
#SBATCH --ntasks=1
#SBATCH --time=10:00

sha1sum /home/llama/firecrest_input_file.txt

For our first job submission we will use two API calls:

1. We will create the FirecREST task of the job submission.
   The definition of the request is [here](http://148.187.97.201:8000/#/Compute/post_compute_jobs).
2. We check the status of the task with a second [request](http://148.187.97.201:8000/#/Tasks/get_tasks__taskid_). If the job is submitted correctly we can get the slurm jobid of the job.



A `POST` request in the `/compute/jobs` endpoint you can see 


The first call is the following:

#### Cell 2C

In [ ]:
machine = 'cluster'
localPath = 'files/firecrest_script.sh'

response = requests.post(
    url=f'{FIRECREST_IP}/compute/jobs',
    headers={'Authorization': f'Bearer {TOKEN}',
             'X-Machine-Name': machine},
    files={'file': open(localPath, 'rb')}
)

handle_response(response)

If everything went well you should get the message `Task created` in the json response. This does **not** necessarily mean that your job is created successfully. This only means that the FirecREST task was created.

Before running the next cell, copy the taskid from the output and set it correctly. In python it should be a string so don't forget the quotes around the task ID.

#### Cell 2D

In [ ]:
taskid = # Fill this assignment with the correct task_id

response = requests.get(
    url=f'{FIRECREST_IP}/tasks/{taskid}',
    headers={'Authorization': f'Bearer {TOKEN}'}
)

handle_response(response)

If your submission was successful you should get the slurm information in the "data" field.

**All the other fields of the json response are about the FirecREST task and not the scheduler.**

### Exercise:

1. If you want information about all the tasks of your user, not a specific `task_id` then you should make a call to the `/tasks/` endpoint, without any path parameter. Try to fill the request on your own to get information about all you tasks.

<a id='all-tasks-cell'></a>
#### Cell 2E

In [ ]:
response = requests.get(
    headers={'Authorization': f'Bearer {TOKEN}'},
    url= # Fill this assignment with the correct endpoint
)

handle_response(response)

Run the next cell to get the solution.

#### Cell 2F

In [ ]:
%cat solutions/all_tasks.py

## Check for the job's status

As soon as we get the slurm job ID, we can get more information on the progress of that job. The call to the `/compute/jobs/{jobid}` endpoint is going to start a FirecREST task for that purpose.

Just like with the job submission, this is a two-calls process:

1. Make a call to FirecREST to make a new task and get the task's ID.
2. Make a call to FirecREST with this task ID to see its results.

Before running this cell you should set the jobid to the jobid of the job you want to test.

#### Cell 2G

In [ ]:
machine = 'cluster'
jobid = # Fill this assignment with the correct jobid

response = requests.get(
    url=f'{FIRECREST_IP}/compute/jobs/{jobid}',
    headers={'Authorization': f'Bearer {TOKEN}',
             'X-Machine-Name': machine}
)

handle_response(response)

# response.ok will be True if no error occured
if response.ok:
    taskid = response.json()['task_id']
    
    print(f"\n{50*'.'}")
    time.sleep(1)
    
    response = requests.get(
        url=f'{FIRECREST_IP}/tasks/{taskid}',
        headers={'Authorization': f'Bearer {TOKEN}'}
    )

    handle_response(response)

### Exercises:

1. Try to explain why you (probably) got this error in the second call: `slurm_load_jobs error: Invalid job id specified`
2. Try to remove the sleep from the last cell and see what happens.

Run the next cell to get a hint for question 1.
The second cell includes the solution to both answers.

#### Cell 2H

In [ ]:
%cat solutions/invalid_id_hint.txt

#### Cell 2I

In [ ]:
%cat solutions/job_status.txt

## Job persistent accounting information

If you want persistent information for older jobs you should make a request in the `compute/acct` endpoint like below.

#### Cell 2J

In [ ]:
machine = 'cluster'
jobid = # Fill this assignment with the correct jobid

response = requests.get(
    url=f'{FIRECREST_IP}/compute/acct',
    headers={'Authorization': f'Bearer {TOKEN}',
             'X-Machine-Name': machine},
    params={'jobs': f'{jobid}'}
)

handle_response(response)

# response.ok will be True if no error occured
if response.ok:
    taskid = response.json()['task_id']
    
    print(f"\n{50*'.'}")
    time.sleep(1)
    
    response = requests.get(
        url=f'{FIRECREST_IP}/tasks/{taskid}',
        headers={'Authorization': f'Bearer {TOKEN}'}
    )

    handle_response(response)

### Job output

When FirecREST submits a job on behalf of the user, a directory will be created in the `$HOME` directory of the user in the machine.

The directory is named `firecrest` and its subdirectories are named after the task IDs of the job submissions. In these subdirectories the user can see the job script that was used for the submission as well as the output file(s) if their location is not specified.